In [32]:
from sklearn.linear_model import Lasso 

import pandas as pd
from readii.io.loaders import loadImageDatasetConfig, loadFileToDataFrame
from pathlib import Path

In [37]:
# Load in the configuration file
config = loadImageDatasetConfig("HEAD-NECK-RADIOMICS-HN1", Path("../../config/datasets"))

# Initialize dataset parameters
CLINICAL_DATA_FILE = config["CLINICAL_FILE"]
DATASET_NAME = config["DATA_SOURCE"] + "_" + config["DATASET_NAME"]

RANDOM_SEED = 10

# general data directory path setup
DATA_DIR_PATH = Path("../../data")
RAW_DATA_PATH = DATA_DIR_PATH / "rawdata" / DATASET_NAME
PROC_DATA_PATH = DATA_DIR_PATH / "procdata" / DATASET_NAME
RESULTS_DATA_PATH = DATA_DIR_PATH / "results" / DATASET_NAME

pyradiomics_settings = "pyradiomics_original_all_features"

In [34]:
# Load clinical data file
clinical_data = loadFileToDataFrame((RAW_DATA_PATH / "clinical" / CLINICAL_DATA_FILE))

In [35]:
# Load the Med-ImageTools index to use for mapping TCIA IDs to local file names
mit_index = loadFileToDataFrame((RAW_DATA_PATH / "images" / config["MIT_INDEX_FILE"]))

# SampleID is local file name
# PatientID is TCIA ID

id_map = mit_index[["SampleID", "PatientID"]].copy()

In [36]:
len(id_map['PatientID'].unique())

82

In [38]:
image_type = "full_original"
pyrad_features = loadFileToDataFrame((RESULTS_DATA_PATH / pyradiomics_settings / f"{image_type}_features.csv"))